# NYC OpenData - Department of Homelessness Analysis

#### Sources

priority | dataset  | description | link |
--------------|--------------|-------------|---------------|
1 | DHS Daily Report | This dataset includes the daily number of families and individuals residing in the Department of <br> Homeless Services (DHS) shelter system and the daily number of families applying to the DHS shelter system. | https://dev.socrata.com/foundry/data.cityofnewyork.us/k46n-sa2m |
2 | Evictions | This dataset lists pending, scheduled and executed evictions within the five boroughs, for the year 2017 - Present. <br> Eviction data is compiled from the majority of New York City Marshals.  | https://dev.socrata.com/foundry/data.cityofnewyork.us/6z8x-wfk4 |
3 | DYCD Demographics by Zip Code | This dataset provides a Demographic breakdown of only DYCD-funded participants within a Zip Code of NYC | https://dev.socrata.com/foundry/data.cityofnewyork.us/hebw-6hze |



#### References

| source  | link |
|--------------|---------------|
| Time Series | https://dev.socrata.com/blog/2019/10/07/time-series-analysis-with-jupyter-notebooks-and-socrata.html |


## Imports


In [ ]:
import dash
import dash
from dash import Dash, dcc, html, Input, Output, dash_table, callback
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
import pandas as pd
from sodapy import Socrata
import plotly.express as px
import nbformat
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from datetime import *
import statsmodels.api as sm
import seaborn as sns
import plotly.io as pio
import numpy as np
from sklearn.linear_model import LinearRegression

## Read data from API to Dataframes

**Dept of Homelessness Daily Report**

In [ ]:
# https://dev.socrata.com/foundry/data.cityofnewyork.us/k46n-sa2m
client = Socrata("data.cityofnewyork.us", None)
results = client.get("k46n-sa2m",limit=80000)

# Convert to pandas DataFrame
dhs_daily_df = pd.DataFrame.from_records(results)

**Evictions**

In [ ]:
# https://dev.socrata.com/foundry/data.cityofnewyork.us/6z8x-wfk4
client = Socrata("data.cityofnewyork.us", None)
results = client.get("6z8x-wfk4",limit=80000)

# Convert to pandas DataFrame
evictions_df = pd.DataFrame.from_records(results)

**DYCD - Dept of Youth & Community Development** - 
Demographics by Zip Code

In [ ]:
# https://dev.socrata.com/foundry/data.cityofnewyork.us/hebw-6hze

client = Socrata("data.cityofnewyork.us", None)
results = client.get("hebw-6hze",limit=80000)


# Convert to pandas DataFrame
demos_by_zip_df = pd.DataFrame.from_records(results)

## Exploratory Data Analysis

### Schema Info + Data Cleaning

#### dhs_report

In [ ]:
# dhs_daily_report
# columns are objects > causing issues downstream when trying to graph > need to convert to numeric


#converting to type numeric
cols = dhs_daily_df.columns.drop('date_of_census')
dhs_daily_df[cols] = dhs_daily_df[cols].apply(pd.to_numeric)
dhs_daily_df.info()

In [ ]:
dhs_daily_df['date_of_census'] = pd.to_datetime(dhs_daily_df['date_of_census'])
dhs_daily_df

#### evictions

In [ ]:
#evictions
#evictions_df.info()
evictions_df['executed_date'] = pd.to_datetime(evictions_df['executed_date'])
evictions_df.sort_values(by='executed_date',ascending=False)

#### demos by zip

In [ ]:
#demos_by_zip_df.info()
demos_by_zip_df['data_os_of_date'] = pd.to_datetime(demos_by_zip_df['data_os_of_date'])
demos_by_zip_df

### Handling Nulls

In [ ]:
# check for nulls 
evictions_df.isna().any()

In [ ]:
# check for nulls 
dhs_daily_df.isna().any()

In [ ]:
demos_by_zip_df.isna().any()

### Data Pivots

#### DHS Time Series Analysis

Questions to Answer <br>
- How has the homeless population in NYC evolved over time?

In [ ]:
dhs_daily_df

In [ ]:
# get percentage of total per subgroup
# denominator is always total_individuals_in_shelter
# create a for loop for every column except total_individuals_in_shelter, divide & then append result as column name + concat perc_ as prefix

# Create a list to store the new column names
new_columns = []

# Iterate over each column
for column in dhs_daily_df.columns:
    if column != 'total_individuals_in_shelter' and column != 'date_of_census':
        # Generate the new column name with the prefix "perc_" followed by the original column name
        new_column = 'perc_' + column
        
        # Divide the values in the current column by the values in the 'total_individuals_in_shelter' column
        new_values = ((dhs_daily_df[column].astype(float) / dhs_daily_df['total_individuals_in_shelter'].astype(float))*100).round(2)
        
        # Append the new column to the DataFrame
        dhs_daily_df[new_column] = new_values
        
        # Append the new column name to the list
        new_columns.append(new_column)

In [ ]:
dhs_daily_df.columns

In [ ]:
'''filtered_columns = []
for column in dhs_daily_df.columns:
    if column.startswith('perc_'):
        filtered_columns.append(column)  
dhs_daily_df[filtered_columns].info()'''

In [ ]:
# convert to monthly & quarterly
# Set the 'date' column as the index
dhs_daily_df.set_index('date_of_census', inplace=True)

# Resample to monthly average data
monthly_avg_df = dhs_daily_df.resample('M').mean()

# Resample to quarterly average data
quarterly_avg_df = dhs_daily_df.resample('Q').mean()

# Resample to quarterly average data
yearly_avg_df = dhs_daily_df.resample('Y').mean()

In [ ]:
#dhs_daily_df
#monthly_avg_df
#quarterly_avg_df
#yearly_avg_df

dhs_daily_df


In [ ]:
monthly_avg_df = monthly_avg_df.filter(regex='^perc_')
#monthly_avg_df

In [ ]:
quarterly_avg_df = quarterly_avg_df.filter(regex='^perc_')
#quarterly_avg_df

In [ ]:
yearly_avg_df = yearly_avg_df.filter(regex='^perc_')
#yearly_avg_df

#### Evictions

In [ ]:
counts_by_boro_piv = pd.pivot_table(evictions_df,values='docket_number', aggfunc='count',index='executed_date',columns='borough').reset_index().sort_values(by='executed_date',ascending=False)
#counts_by_boro_piv = pd.pivot_table(results_df,values='docket_number', aggfunc='count',index='executed_date',columns='borough')

In [ ]:
# Convert 'executed_date' column to datetime
counts_by_boro_piv['executed_date'] = pd.to_datetime(counts_by_boro_piv['executed_date'])

In [ ]:
counts_by_boro_piv

### Data Visualizations

#### evictions

In [ ]:
# Melt the DataFrame
counts_by_boro_fig = counts_by_boro_piv.melt(id_vars='executed_date', var_name='borough', value_name='count')

In [ ]:
# Create bar plot using Plotly
fig = px.bar(counts_by_boro_fig, x='executed_date', y='count', color='borough', barmode='group')

# Display the plot
fig.show()

#### dept homelessness

##### time series

In [ ]:
dhs_daily_df.info()
#monthly_avg_df.info()
#quarterly_avg_df.info()

In [ ]:
# filter functions

# date filter
def get_date(date_of_census, start_date, end_date):
    date_of_census=date_of_census
    start_date=start_date
    end_date=end_date
    return start_date <= date_of_census <= end_date

# column filter
def get_columns(df, desired_columns):
    columns = []
    for i, column in enumerate(df.columns):
        if i in desired_columns:
            columns.append(column)
    return df[columns]

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filter = get_columns(dhs_daily_df,desired_columns)
#filter.columns
dhs_daily_df.columns

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(dhs_daily_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the bar chart
fig = px.line(filtered_data, x='date_of_census', y='total_individuals_in_shelter',
             color_discrete_sequence=['orange',],
             labels=dict(
                 date_of_census='Year Start',
                 total_individuals_in_shelter='Total Sheltered Individuals'
             ),             
             title='Daily Total Individuals in NYC Shelters',            
             )

# Customize the layout if needed
fig.update_layout( xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0),                  
                  yaxis=dict(tickmode='auto', tickformat='d', range=[40000, filtered_data['total_individuals_in_shelter'].max()]),
                  margin=dict(l=100, r=50, t=80, b=20),  # Adjust the left and right margin values as desired
                  width=1500,  # Set the desired width of the figure
                  height=500  # Set the desired height of the figure
                  )


# Display the plot
fig.show()

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(dhs_daily_df,desired_columns)
filtered_data.columns

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10,11]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(dhs_daily_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the bar chart
fig = px.line(filtered_data, x='date_of_census', y=['total_individuals_in_families_with_children_in_shelter_','total_single_adults_in_shelter','individuals_in_adult_families_in_shelter'],
             color_discrete_sequence=['sienna', 'orange','silver'],
             labels=dict(
                 date_of_census='Year Start',
                 total_individuals_in_shelter='Total Sheltered Individuals'
             ),             
             title='Daily Total Individuals in NYC Shelters',            
             )

# Customize the layout if needed
fig.update_layout( xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.0, xanchor='left', x=0),                  
                  yaxis=dict(tickmode='auto', tickformat='d', range=[0, filtered_data['total_individuals_in_shelter'].max()]),
                  margin=dict(l=100, r=50, t=80, b=20),  # Adjust the left and right margin values as desired
                  width=1500,  # Set the desired width of the figure
                  height=500  # Set the desired height of the figure
                  )


# Display the plot
fig.show()

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(monthly_avg_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the bar chart
fig = px.line(filtered_data, x='date_of_census', y=['perc_total_adults_in_shelter', 'perc_total_children_in_shelter','perc_total_single_adults_in_shelter','perc_total_individuals_in_families_with_children_in_shelter_',],
             color_discrete_sequence=['silver','purple','orange', 'sienna'],
             labels={'date_of_census': 'Year_Start', 'value': 'Percentage'},
             title='Average Population in NYC Shelters (%)',
             
             )

# Customize the layout if needed
fig.update_layout( xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0),
                  xaxis=dict(tickmode='auto', tickformat='%Y'),
                  width=1500,  # Set the desired width of the figure
                  height=500  # Set the desired height of the figure
                  )

# Display the plot
fig.show()

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(monthly_avg_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the line chart
fig = px.line(filtered_data, x='date_of_census', y=['perc_total_adults_in_shelter', 'perc_total_children_in_shelter'],
             color_discrete_sequence=['silver', 'orange'],
             labels={'date_of_census': 'Year_Start', 'value': 'Percentage'},
             title='Average Population in NYC Shelters (%)',
             width=1500,  # Set the desired width of the figure
             height=500  # Set the desired height of the figure
             )

# Add the second y-axis
fig.update_traces(yaxis="y")

# Add data labels
texts = [filtered_data['perc_total_adults_in_shelter'].apply(lambda x: f'{round(x)}%'),
         filtered_data['perc_total_children_in_shelter'].apply(lambda x: f'{round(x)}%')]
for i, t in enumerate(texts):
    fig.data[i].text = t

# Customize the layout if needed
fig.update_layout(xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0),
                  xaxis=dict(tickmode='auto', tickformat='%Y'),
                  yaxis=dict(title='Percentage', side='left'),  # Set side='left' for the first y-axis
                  )

# Display the plot
fig.show()


##### distribution

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(yearly_avg_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the bar chart
fig = px.bar(filtered_data, x='date_of_census', y=['perc_total_individuals_in_families_with_children_in_shelter_','perc_total_single_adults_in_shelter','perc_individuals_in_adult_families_in_shelter'],
             color_discrete_sequence=['sienna', 'orange','silver'],
             labels={'date_of_census': 'Year_Start', 'value': 'Percentage'},
             title='NYC Sheltered Population Year-over-Year',
             width=1500,  # Set the desired width of the figure
            height=500  # Set the desired height of the figure
             )

# Customize the layout if needed
fig.update_layout(barmode='group', xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0))

# Add data labels
texts = [filtered_data['perc_total_individuals_in_families_with_children_in_shelter_'].apply(lambda x: f'{round(x)}%'),
         filtered_data['perc_total_single_adults_in_shelter'].apply(lambda x: f'{round(x)}%'),
         filtered_data['perc_individuals_in_adult_families_in_shelter'].apply(lambda x: f'{round(x)}%'),
         ]
for i, t in enumerate(texts):
    fig.data[i].text = t
fig.update_traces(textposition='inside', textfont=dict(color='black', size=10), textangle=0)

# Display the plot
fig.show()

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(yearly_avg_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the bar chart
fig = px.bar(filtered_data, x='date_of_census', y=['perc_total_adults_in_shelter', 'perc_total_children_in_shelter'],
             color_discrete_sequence=['silver', 'orange'],
             labels={'date_of_census': 'Year_Start', 'value': 'Percentage'},
             title='Percentage of Adults vs. Children in NYC Shelter YoY',
             
             )

# Customize the layout if needed
fig.update_layout(barmode='group', xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0),
                  xaxis=dict(tickmode='auto', tickformat='%Y'),
                  width=1500,  # Set the desired width of the figure
                  height=500  # Set the desired height of the figure
                  )

# Add data labels
texts = [filtered_data['perc_total_adults_in_shelter'].apply(lambda x: f'{round(x)}%'),
         filtered_data['perc_total_children_in_shelter'].apply(lambda x: f'{round(x)}%')]
for i, t in enumerate(texts):
    fig.data[i].text = t
fig.update_traces(textposition='inside', textfont=dict(color='black', size=10), textangle=0)

# Display the plot
fig.show()

In [ ]:
desired_columns = [0,1,2,3,4,5,6,7,8,9,10]
start_date = datetime(2013, 1, 1)
end_date = datetime(2022, 12, 31)

# Convert index values to datetime
filtered_data = get_columns(yearly_avg_df,desired_columns)

# Apply date filter
filtered_data = filtered_data[filtered_data.index.to_series().apply(get_date, start_date=start_date, end_date=end_date)]
filtered_data.reset_index(inplace=True)
filtered_data

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"

# Create the bar chart
fig = px.bar(filtered_data, x='date_of_census', y=['perc_single_adult_men_in_shelter', 'perc_single_adult_women_in_shelter'],
             color_discrete_sequence=['sienna', 'silver'],
             labels={'date_of_census': 'Year_Start', 'value': 'Percentage'},
             title='NYC Sheltered Population by Gender YoY',
             
             )

# Customize the layout if needed
fig.update_layout(barmode='group', xaxis_tickangle=0,
                  legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0),
                  width=1500,  # Set the desired width of the figure
                  height=500  # Set the desired height of the figure
                  )


# Add data labels
texts = [filtered_data['perc_single_adult_men_in_shelter'].apply(lambda x: f'{round(x)}%'),
         filtered_data['perc_single_adult_women_in_shelter'].apply(lambda x: f'{round(x)}%')]
for i, t in enumerate(texts):
    fig.data[i].text = t
fig.update_traces(textposition='inside', textfont=dict(color='black', size=10), textangle=0)

# Display the plot
fig.show()

##### correlation analyses

In [ ]:
'''correlation_matrix = filtered_data.corr()  # Correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='rocket_r')
plt.show()'''

In [ ]:
'''desired_columns = [0,1,2,3,4,5,6,7,8]
filtered_data = get_columns(quarterly_avg_df,desired_columns)


correlation_matrix = filtered_data.corr()  # Correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='rocket_r')
plt.show()'''

#### web app development

In [ ]:
app = dash.Dash(__name__)

date_range_select = dcc.DatePickerRange(
        id="date-range",
        min_date_allowed=datetime(2013, 1, 1),
        max_date_allowed=datetime.today(),
        start_date=datetime(2013, 1, 1),
        end_date=datetime.today(),
    )

output = dcc.Graph(style={'width':'90vw',
                          'height':'50vw'},
                   id="graph")

app.layout = html.Div([
    dbc.Row(date_range_select),
    dbc.Row(output),
])

@app.callback(
    Output("graph", "figure"),
    Input("date-range", "start_date"),
    Input("date-range", "end_date")
)
def data_visualization(start_date, end_date):
    filtered_data = get_columns(dhs_daily_df, desired_columns)

    # Apply date filter
    filtered_data = filtered_data[
        (filtered_data.index >= start_date) & (filtered_data.index <= end_date)
    ]

    # Set the template to 'plotly_dark'
    pio.templates.default = "plotly_dark"

    # Create the line chart
    fig = px.line(
        filtered_data,
        x=filtered_data.index,
        y='total_individuals_in_shelter',
        color_discrete_sequence=['orange'],
        labels=dict(
            x='Date',
            y='Total Sheltered Individuals'
        ),
        title='Daily Total Individuals in NYC Shelters'
    )

    # Customize the layout if needed
    fig.update_layout(
        xaxis_tickangle=0,
        legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='left', x=0),
        yaxis=dict(tickmode='auto', tickformat='d', range=[40000, filtered_data['total_individuals_in_shelter'].max()]),
        margin=dict(l=100, r=50, t=80, b=20),  # Adjust the left and right margin values as desired
        width=1500,  # Set the desired width of the figure
        height=500  # Set the desired height of the figure
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

# forecasting

In [ ]:
# forecasting
def forecast_sarimax(time_series, forecast_steps):
    forecasts = {}

    for column in time_series.columns:
        # Extract the individual time series
        ts = time_series[column]

        # Fit the SARIMAX model
        model = sm.tsa.SARIMAX(ts, order=(1, 0, 0), seasonal_order=(0, 0, 0, 0))
        model_fit = model.fit()

        # Forecast future values
        forecast = model_fit.get_forecast(steps=forecast_steps)

        # Extract the forecasted values and confidence intervals
        forecast_values = forecast.predicted_mean
        forecast_ci = forecast.conf_int()

        # Create a DataFrame with the forecasted values and confidence intervals
        forecast_df = pd.DataFrame({'Forecast': forecast_values,
                                    'Lower CI': forecast_ci.iloc[:, 0],
                                    'Upper CI': forecast_ci.iloc[:, 1]})

        # Store the forecast for the current variable
        forecasts[column] = forecast_df

    return forecasts

In [ ]:
'''forecasts = forecast_sarimax(filtered_data, forecast_steps=5)
variable_name = 'perc_total_adults_in_shelter'
forecast_variable_1 = forecasts[variable_name]
forecast_variable_1'''